In [45]:
import os

NEO4J_URI = os.environ['NEO4J_URI']
NEO4J_USERNAME = os.environ['NEO4J_USERNAME']
NEO4J_PASSWORD = os.environ['NEO4J_PASSWORD']

In [46]:
from neo4j import GraphDatabase
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [47]:
def genMatchDefinedLabelInferenceQueries():
    records, summary, keys = driver.execute_query(
        """MATCH (mdl:MatchDefinedLabel)
        RETURN mdl.name AS name, mdl.pattern AS pattern, mdl.classElementVariable AS classElementVariable""",
        database_="ontology",
    )
    for record in records:
        query = f"MATCH {record['pattern']} SET {record['classElementVariable']}:{record['name']}"
        yield query

for q in genMatchDefinedLabelInferenceQueries():
    print(q)


MATCH (p:Person) WHERE EXISTS {(p)-[:DIRECTED]->()} SET p:_PersonDirectedSome
MATCH (p:Person {name: 'Kevin Bacon'}) SET p:_KevinBacon
MATCH (p:Person) WHERE EXISTS {(p)-[:ACTED_IN]->()} SET p:_PersonActedInSome


In [48]:
def genMatchDefinedRelationshipInferenceQueries():
    records, summary, keys = driver.execute_query(
        """MATCH (mdr:MatchDefinedRelationship)
        RETURN mdr.name AS name, mdr.pattern AS pattern, mdr.sourceElementVariable AS sourceElementVariable, mdr.targetElementVariable AS targetElementVariable""",
        database_="ontology",
    )
    for record in records:
        query = f"MATCH {record['pattern']} MERGE ({record['sourceElementVariable']})-[:{record['name']}]->({record['targetElementVariable']})"
        yield query

for q in genMatchDefinedRelationshipInferenceQueries():
    print(q)


MATCH (s:Person)-[:INVOLVED_IN]->()<-[:INVOLVED_IN]-(t:Person) MERGE (s)-[:_COLLABORATOR]->(t)
MATCH (s:Person)-[:ACTED_IN]->()<-[:ACTED_IN]-(t:Person) MERGE (s)-[:_COACTOR]->(t)


In [49]:
def genSCOLabelInferenceQueries():
    records, summary, keys = driver.execute_query(
        """MATCH (narrower:Label)-[:SCO]->(broader:Label)
        RETURN narrower.name AS narrower, broader.name AS broader""",
        database_="ontology",
    )
    for record in records:
        query = f"MATCH (n:{record['narrower']}) SET n:{record['broader']}"
        yield query

for q in genSCOLabelInferenceQueries():
    print(q)

MATCH (n:_KevinBacon) SET n:Actor
MATCH (n:Actor) SET n:Person
MATCH (n:Director) SET n:Person


In [50]:
def genImpliesRelationshipInferenceQueries():
    records, summary, keys = driver.execute_query(
        """MATCH (narrower:Relationship)-[:IMPLIES]->(broader:Relationship)
        RETURN narrower.name AS narrower, broader.name AS broader""",
        database_="ontology",
    )
    for record in records:
        query = f"MATCH (n)-[:{record['narrower']}]->(m) MERGE (n)-[:{record['broader']}]->(m)"
        yield query

for q in genImpliesRelationshipInferenceQueries():
    print(q)

MATCH (n)-[:ACTED_IN]->(m) MERGE (n)-[:INVOLVED_IN]->(m)


In [51]:
def genEquivalentLabelInferenceQueries():
    records, summary, keys = driver.execute_query(
        """MATCH (l1:Label)-[:EQUIVALENT]->(l2:Label)
        RETURN l1.name AS l1, l2.name AS l2""",
        database_="ontology",
    )
    for record in records:
        query = f"MATCH (n:{record['l1']}|{record['l2']}) SET n:{record['l1']}:{record['l2']}"
        yield query

for q in genEquivalentLabelInferenceQueries():
    print(q)

MATCH (n:Actor|_PersonActedInSome) SET n:Actor:_PersonActedInSome
MATCH (n:Director|_PersonDirectedSome) SET n:Director:_PersonDirectedSome


In [52]:
def genEquivalentRelationshipInferenceQueries():
    records, summary, keys = driver.execute_query(
        """MATCH (r1:Relationship)-[:EQUIVALENT]->(r2:Relationship)
        RETURN r1.name AS r1, r2.name AS r2""",
        database_="ontology",
    )
    for record in records:
        query = f"MATCH (n)-[:{record['r1']}|{record['r2']}]->(m) MERGE (n)-[:{record['r1']}]->(m) MERGE (m)-[:{record['r1']}]->(n)"
        yield query

for q in genEquivalentRelationshipInferenceQueries():
    print(q)

MATCH (n)-[:COLLABORATOR|_COLLABORATOR]->(m) MERGE (n)-[:COLLABORATOR]->(m) MERGE (m)-[:COLLABORATOR]->(n)
MATCH (n)-[:COACTOR|_COACTOR]->(m) MERGE (n)-[:COACTOR]->(m) MERGE (m)-[:COACTOR]->(n)


In [53]:
def genSymmetricRelationshipInferenceQueries():
    records, summary, keys = driver.execute_query(
        """MATCH (r:Relationship&Symmetric)
        RETURN r.name AS sim_rel""",
        database_="ontology",
    )
    for record in records:
        query = f"MATCH (n)-[:{record['sim_rel']}]->(m) MERGE (m)-[:{record['sim_rel']}]->(n)"
        yield query

for q in genSymmetricRelationshipInferenceQueries():
    print(q)

MATCH (n)-[:_COLLABORATOR]->(m) MERGE (m)-[:_COLLABORATOR]->(n)
MATCH (n)-[:COLLABORATOR]->(m) MERGE (m)-[:COLLABORATOR]->(n)
MATCH (n)-[:COACTOR]->(m) MERGE (m)-[:COACTOR]->(n)
MATCH (n)-[:_COACTOR]->(m) MERGE (m)-[:_COACTOR]->(n)


In [54]:
def genMatchDefinedNodePropertyInferenceQueries():
    records, summary, keys = driver.execute_query(
        """MATCH (n:Label)-[:HAS_PROPERTY]->(qdp:MatchDefinedNodeProperty)
        RETURN n.name AS label, qdp.name AS property_name, qdp.value AS property_value, qdp.propertyOwnerVariable AS variable, qdp.valueVariable AS val_variable""",
        database_="ontology",
    )
    for record in records:
        query = f"""MATCH ({record['variable']}:{record['label']})
        CALL ({record['variable']}) {{ {record['property_value']}
        WHERE {record['variable']}.{record['property_name']} IS NULL OR {record['variable']}.{record['property_name']} <> {record['val_variable']}
        SET {record['variable']}.{record['property_name']} = {record['val_variable']} }}
        IN CONCURRENT TRANSACTIONS OF 10 ROWS"""
        yield query

for q in genMatchDefinedNodePropertyInferenceQueries():
    print(q)

MATCH (x:Actor)
        CALL (x) { MATCH SHORTEST 1 (x)-[ca:COACTOR]-*(y:_KevinBacon) WITH ca LIMIT 1 WITH size(ca) AS kbn
        WHERE x.kb_number IS NULL OR x.kb_number <> kbn
        SET x.kb_number = kbn }
        IN CONCURRENT TRANSACTIONS OF 10 ROWS


In [55]:
inferenceRulesGenerators = [genMatchDefinedLabelInferenceQueries,
                            genMatchDefinedRelationshipInferenceQueries,
                            genSCOLabelInferenceQueries,
                            genImpliesRelationshipInferenceQueries,
                            genEquivalentLabelInferenceQueries,
                            genEquivalentRelationshipInferenceQueries,
                            genSymmetricRelationshipInferenceQueries,
                            genMatchDefinedNodePropertyInferenceQueries]

def genOntologyInferenceQueries(inferenceRulesGenerators):
    for rule in inferenceRulesGenerators:
        for q in rule():
            yield q

list(genOntologyInferenceQueries(inferenceRulesGenerators))

['MATCH (p:Person) WHERE EXISTS {(p)-[:DIRECTED]->()} SET p:_PersonDirectedSome',
 "MATCH (p:Person {name: 'Kevin Bacon'}) SET p:_KevinBacon",
 'MATCH (p:Person) WHERE EXISTS {(p)-[:ACTED_IN]->()} SET p:_PersonActedInSome',
 'MATCH (s:Person)-[:INVOLVED_IN]->()<-[:INVOLVED_IN]-(t:Person) MERGE (s)-[:_COLLABORATOR]->(t)',
 'MATCH (s:Person)-[:ACTED_IN]->()<-[:ACTED_IN]-(t:Person) MERGE (s)-[:_COACTOR]->(t)',
 'MATCH (n:_KevinBacon) SET n:Actor',
 'MATCH (n:Actor) SET n:Person',
 'MATCH (n:Director) SET n:Person',
 'MATCH (n)-[:ACTED_IN]->(m) MERGE (n)-[:INVOLVED_IN]->(m)',
 'MATCH (n:Actor|_PersonActedInSome) SET n:Actor:_PersonActedInSome',
 'MATCH (n:Director|_PersonDirectedSome) SET n:Director:_PersonDirectedSome',
 'MATCH (n)-[:COLLABORATOR|_COLLABORATOR]->(m) MERGE (n)-[:COLLABORATOR]->(m) MERGE (m)-[:COLLABORATOR]->(n)',
 'MATCH (n)-[:COACTOR|_COACTOR]->(m) MERGE (n)-[:COACTOR]->(m) MERGE (m)-[:COACTOR]->(n)',
 'MATCH (n)-[:_COLLABORATOR]->(m) MERGE (m)-[:_COLLABORATOR]->(n)',
 'M

In [56]:
def infer (rules, params={}):
    """
    This is a function you can use if you want to run a set of inference rules
    until a convergence is reached. why not use it in a RDF-like reasoning context?
    """
    counter = 0
    while True:
        counter += 1
        any_update = False
        for rule in rules:
            with driver.session(database="neo4j") as session:
                result = session.run(rule, params)
            any_new_update = result.consume().counters._contains_updates
            any_update = any_update or any_new_update
        if not any_update:
            break

In [57]:
infer(list(genOntologyInferenceQueries(inferenceRulesGenerators)))